In [1]:
import os
import numpy as np
from data_generator import DataGenerator

In [2]:
label_name = {}
for index, folder in enumerate(os.listdir(f'data/train')):
    label_name[folder] = index-1

In [3]:
train_paths = np.array([])
train_labels = {}
for folder in os.listdir('data/train'):
    for file in os.listdir(f'data/train/{folder}'):
        # required due to .DS_Store
        if file[0] == '.':
            continue
        train_paths = np.append(train_paths, np.array([f'data/train/{folder}/{file}']))
        train_labels[f'data/train/{folder}/{file}'] = label_name[folder]

validate_paths = np.array([])
validate_labels = {}
for folder in os.listdir('data/validation'):
    for file in os.listdir(f'data/validation/{folder}'):
        if file[0] == '.':
            continue
        validate_paths = np.append(validate_paths, np.array([f'data/validation/{folder}/{file}']))
        validate_labels[f'data/validation/{folder}/{file}'] = label_name[folder]

In [4]:
train_generator = DataGenerator(train_paths, train_labels, batch_size=128, n_channels=3)
validate_generator = DataGenerator(validate_paths, validate_labels, batch_size=128, n_channels=3)

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, GlobalMaxPooling2D

model = Sequential([
    Conv2D(filters=16, kernel_size=(3,3), input_shape=(250,250,3), activation='relu', padding='same'),
    Conv2D(filters=1, kernel_size=(2,2), strides=(2,2)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=1, kernel_size=(2,2), strides=(2,2)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    GlobalMaxPooling2D(),
    Dense(5, activation='softmax')
])

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
os.makedirs('models')

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callback = [ModelCheckpoint(f'models/2.h5', monitor='val_loss', verbose=1, save_best_only=True), EarlyStopping(monitor='val_loss', verbose=1, patience=5)]

In [5]:
from tensorflow.keras.models import load_model
model = load_model('models/2.h5')

In [ ]:
model.fit(train_generator, epochs=100, batch_size=128, validation_data=validate_generator, callbacks=callback)

In [15]:
test_paths = np.array([])
test_labels = {}
for folder in os.listdir('data/test'):
    for file in os.listdir(f'data/test/{folder}'):
        if file[0] == '.':
            continue
        test_paths = np.append(test_paths, np.array([f'data/test/{folder}/{file}']))
        test_labels[f'data/test/{folder}/{file}'] = label_name[folder]

In [16]:
test_generator = DataGenerator(test_paths, test_labels, batch_size=128, n_channels=3)

In [17]:
model.evaluate(test_generator)

58/58 [==============================] - 15s 257ms/step - loss: 0.0268 - accuracy: 0.9919


[0.026802781969308853, 0.9919180870056152]

Test performance differences when MaxPooling is used.

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalMaxPooling2D

model = Sequential([
    Conv2D(filters=16, kernel_size=(3,3), input_shape=(250,250,3), activation='relu', padding='same'),
    MaxPooling2D(),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPooling2D(),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    GlobalMaxPooling2D(),
    Dense(5, activation='softmax')
])

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callback = [ModelCheckpoint(f'models/2_max_pooling.h5', monitor='val_loss', verbose=1, save_best_only=True), EarlyStopping(monitor='val_loss', verbose=1, patience=5)]

In [ ]:
model.fit(train_generator, epochs=100, batch_size=128, validation_data=validate_generator, callbacks=callback)

In [22]:
model.evaluate(test_generator)

58/58 [==============================] - 14s 240ms/step - loss: 0.0404 - accuracy: 0.9875


[0.04041128605604172, 0.9874730706214905]